# Detector versus Discriminator
A **detector** is a circuit that extracts the modulating signal from the modulated carrier.  A **discriminator** is a specific type of detector that converts frequency variations of the modulated signal into amplitude variations, which can then be demodulated to recover the original audio signal.

# Weiss Discriminator
Jessop : p4.32 Receiver, fig. 82 : Weiss discriminator
[praktische vraagjes superhet](https://www.circuitsonline.net/forum/view/86845/2?mode=or&query=fm+antenne)
<img src="./doc/weiss_20110321fet.png" alt="Weiss discriminator" width="640" height="480">
* hard to get a low distortion at the output.  THD below 0.5% is difficult to achieve.
* simple to build, no tapped inductor
* doesn't require 2nd stage mixer, so lower parts count
* requires amplification at 10.7MHz IF, before going into the discriminator

# Pulse Counting Discriminator
## Advantages
* inherently low distortion
* high dynamic range
* a frequency response to DC
* no need for tuned circuits
* no need for amplification at 10.7MHz IF, can amplify at low IF frequency (100kHz or lower).  This simplifies the design of the IF amplifier.

## Disadvantages
* it requires a mixer stage to convert the RF (single conversion superhet)  or IF (double conversion superhet) to a lower IF frequency (e.g. 100kHz)

## Reference
* [Phil's Valve Radio site : Double Conversion Pulse Counting FM Superhet Receiver With 10.7 MHz First IF Stage](https://www.philsvalveradiosite.co.uk/doubleconversionpulsecountingfmtuner_1.htm) :
  * IF1 mixer : 10.7MHz out
  * IF1 mixer directly into a 2nd mixer (no crystal filter on 10.7MHz!): 150kHz out
  * IF2 mixer followed by IF2-amplifier
  * IF2-amplifier followed by pulse counting discriminator
* [W2AEW : #303: What is a Pulse Counting FM Demodulator | Detector | Discriminator? ](https://www.youtube.com/watch?v=jQlN2fc7LJc)
  * single conversion superhet : 100MHz RF, 100.1MHz LO, 100kHz IF -> doesn't reject image, but there aren't none when using a signal generator.
  * 100kHz pulse counting discriminator
  * DBM with 1N4148
* [Alan Yates (VK2ZAY) 2014: Advent Calendar of Circuits 2014: Day 15: Pulse-Counting WBFM Receiver](https://www.youtube.com/watch?v=lP3HltRJV6A)
  * single conversion superhet : 100MHz RF, 100.1MHz LO, 150kHz-180kHz IF
  * J310 JFET mixer
  * 150kHz-180kHz IF amplifier (which will oscillate without proper input signal)
  * Remarks : it's not very sensitive (would be better with parallel RLC at the gate of the mixer), and it's not very selective, but still better than a super-regenerative receiver.
* [Alan Yates (VK2ZAY) 2010: Pulse-Counting FM Broadcast Receiver](http://www.vk2zay.net/article/250), [offline version](./doc/Alan%20Yates'%20Laboratory%20-%20Pulse-Counting%20FM%20Broadcast%20Receiver.pdf)
  * single conversion superhet : 100kHz IF
  * 2N5484 JFET mixer
  * IF amplifier (for 100kHz IF)
* [kvdijken Pulse Counting FM Detector](https://github.com/kvdijken/FM-Receiver/blob/main/documentation/Pulse%20Counting%20FM%20Detector.pdf) : uses a 2N3819 JFET mixer
* [forum](https://www.circuitsonline.net/forum/view/99082#highlight=discriminator)

## Design
Some of the resources use a single JFET mixer, but that has strong dependence on device to device variations, and it leaks the LO signal into the output quite strongly.  A better solution is to use a double balanced mixer, which has a much lower LO leakage and is less sensitive to device variations, as done by W2AEW in his video.  Moreover, we're already using a double balanced mixer in the front-end, so we can use the same type of mixer for the pulse counting discriminator.

# Quadrature Detector
The [NJM14570](https://www.nisshinbo-microdevices.co.jp/en/products/fm-if-demodulator-ic/spec/?product=njm14570) is a quadrature detector that can be used for FM demodulation. It has a built-in IF amplifier that yields 12dB SINAD at -87dBm input signal level, so it can be used as a complete IF stage and we don't need a separate IF amplifier. 
The NJM14570 is designed to work with an IF frequency of 10.7 MHz, making it suitable for FM broadcast applications.  

## Inductor
The NJM14570 can use an IFT or an 2.2µH inductor (e.g. NLV32T-2R2J-PF: 2.2µH, Q>30 @10.7MHz, 5% tolerance) for the tank circuit.

## Trimming Capacitor
* 10pF : SGC3S100, JZ100, STC3MB10-T1 (JLCPCB part number C22468121)

## Reference
* [yo5ofh : Quadrature FM Detectors](https://www.qsl.net/yo5ofh/doc/fm_detectors/quadrature_fm_detectors.htm)
 
# Broadcast FM 
## Audio frequency range
The audio frequency range for FM broadcast is **30 Hz to 15 kHz**. This range is suitable for high-fidelity sound reproduction, allowing for a wide dynamic range and clarity in audio transmission.

## Frequency Deviation ∆𝑓
In FM broadcast, it has been internationally agreed to restrict maximum deviation to **∆𝑓= 75 𝑘𝐻𝑧** on each side of carrier frequency (𝑓𝑐).
Frequency deviation (∆𝑓) is related to LOUDNESS (= AMPLITUDE) of the modulating signal. Louder the sound, greater the (∆𝑓) and vice-versa.

## Modulation Index 𝑚
The modulation index (𝑚) is defined as the ratio of frequency deviation (∆𝑓) to the modulating frequency (𝑓𝑚): 𝑚 = ∆𝑓/𝑓𝑚

## Deviation Ratio 𝑑
The deviation ratio (𝑑) is defined as the ratio of the maximum carrier frequency deviation (∆𝑓) to the maximum modulating frequency (𝑓𝑚): 𝑑 = max(∆𝑓)/max(𝑓𝑚).

For FM broadcast, the maximum modulating frequency is typically 15 kHz, leading to a deviation ratio of:
$$d = \frac{75 \, \text{kHz}}{15 \, \text{kHz}} = 5 $$

# Diplexer
This is only for use in the pulse counting detector design that works at 200kHz IF frequency.

By choosing the right value for the LF, we can arrive at inductor values that are easy to obtain.  If we choose LF too low, then we'll end up with large inductors that have self-resonance below 10MHz, which is not what we want.  

In [12]:
# Third order Low Pass High Pass Diplexer, EMRFD p3.38
import math
f = 220e3 # 100 kHz
ω = 2 * math.pi * f

def calc_L(ω, Z):
    """Calculate inductance for a given ω and impedance."""
    return Z / ω

def calc_C(ω, Z):
    """Calculate capacitance for a given ω and impedance."""
    return 1 / (ω * Z)

L_25 = calc_L(ω, 25)  # L for 25 ohm
C_25 = calc_C(ω, 25)  # C for 25 ohm
L_375 = calc_L(ω, 37.5)  # L for 37.5 ohm
C_375 = calc_C(ω, 37.5)  # C for 37.5 ohm
L_75 = calc_L(ω, 75)  # L for 75 ohm
C_75 = calc_C(ω, 75)  # C for 75 ohm

print(f"(25 ohm): L = {L_25:.6e} H, C = {C_25:.6e} F")
print(f"(37.5 ohm): L = {L_375:.6e} H, C = {C_375:.6e} F")
print(f"(75 ohm): L = {L_75:.6e} H, C = {C_75:.6e} F")

(25 ohm): L = 1.808579e-05 H, C = 2.893726e-08 F
(37.5 ohm): L = 2.712868e-05 H, C = 1.929151e-08 F
(75 ohm): L = 5.425737e-05 H, C = 9.645754e-09 F


<figure>
<img src="./doc/diplexer_simulation.png" alt="Diplexer simulation" width="1000"/>
<figcaption>Diplexer simulation</figcaption>
</figure>

# Audio filter
* [Subwoofer Low Pass Filter using Transistors](https://www.homemade-circuits.com/make-this-low-pass-filter-circuit-for/)
* [Transistor High Pass Filter Circuit](https://www.electronics-notes.com/articles/analogue_circuits/transistor/active-high-pass-filter-circuit.php)